At first, we have to import the train data.


In [1]:
import pandas as pd

df = pd.read_csv('data/train_ship_segmentations_v2.csv').dropna()


Lets take a look at dataframe.

In [2]:
df.head


<bound method NDFrame.head of               ImageId                                      EncodedPixels
2       000155de5.jpg  264661 17 265429 33 266197 33 266965 33 267733...
3       000194a2d.jpg  360486 1 361252 4 362019 5 362785 8 363552 10 ...
4       000194a2d.jpg  51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...
5       000194a2d.jpg  198320 10 199088 10 199856 10 200624 10 201392...
6       000194a2d.jpg  55683 1 56451 1 57219 1 57987 1 58755 1 59523 ...
...               ...                                                ...
231694  fff909871.jpg  441057 2 441825 4 442592 7 443360 9 444128 11 ...
231709  fffcb6464.jpg  222782 4 223550 10 224318 14 225086 14 225854 ...
231712  fffd924fb.jpg  110011 5 110779 5 111547 5 112315 5 113083 5 1...
231713  fffd924fb.jpg  198440 8 199208 8 199976 8 200744 8 201512 8 2...
231714  fffdd2377.jpg  333603 7 334359 19 335120 26 335888 26 336656 ...

[81723 rows x 2 columns]>

Lets look at images from dataset.


In [3]:
import os
from PIL import Image

train_dir = "data/train_v2/"

for filename in os.listdir(train_dir)[:10]:
    with Image.open(train_dir+filename) as im:
        im.show()


The next thing to do is making a run-length decoder, we should remember, that images can have multiple masks.



In [4]:
IMG_SIZE = 768

def rle_to_mask(array, rle):
    '''

    :param array: single image
    :param rle: list of rle masks, for every image
    :return: image, with applied masks
    '''
    if rle == False:
        return array

    for elem in rle:
        list0 = list(map(int, elem.split(" ")))
        elem = list(zip(list0[0::2], list0[1::2]))

        place = []
        for i in elem:
            start, length = i
            coordinate = (start % IMG_SIZE, start // IMG_SIZE, length)
            place.append(coordinate)

        for x, y, l in place:
            for i in range(0, l):
                array[x-1+i, y-1] = 255
    return array




The work of decoder can be seen below:


In [7]:
import numpy as np

for filename in os.listdir(train_dir)[:10]:
    masks = list(df.loc[df['ImageId'] == filename, 'EncodedPixels'])
    zero = np.zeros((IMG_SIZE, IMG_SIZE))
    pixels = rle_to_mask(zero, masks)
    PIL_image = Image.fromarray(pixels.astype('uint8'), 'L')
    PIL_image.show()
